In [ ]:
import unittest
import pandas as pd
from pandas.testing import assert_frame_equal
import os
from gff_to_df import gff_to_df  # Import the function to be tested
import io

class TestGffToDf(unittest.TestCase):

    def setUp(self):
        # Create a temporary directory for test files
        self.test_dir = "temp_test_dir"
        os.makedirs(self.test_dir, exist_ok=True)

    def tearDown(self):
        # Clean up the temporary directory and files after tests
        for file in os.listdir(self.test_dir):
            os.remove(os.path.join(self.test_dir, file))
        os.rmdir(self.test_dir)

    def create_dummy_gff(self, file_path, content):
        with open(file_path, 'w') as f:
            f.write(content)

    def test_gff_to_df_valid(self):
        """Test with a valid GFF file."""
        valid_gff_content = """##gff-version   3
# seqname	source	feature	start	end	score	strand	frame	attribute
chr1	test	gene	1	100	.	+	.	ID=gene1;Name=test_gene
chr1	test	mRNA	10	90	.	+	.	Parent=gene1;ID=mrna1
chr1	test	exon	20	50	.	+	.	Parent=mrna1
"""
        file_path = os.path.join(self.test_dir, "ref_query.gff")
        self.create_dummy_gff(file_path, valid_gff_content)

        expected_df = pd.DataFrame({
            'seq_id': ['chr1', 'chr1', 'chr1'],
            'source': ['test', 'test', 'test'],
            'type': ['gene', 'mRNA', 'exon'],
            'start': [1, 10, 20],
            'end': [100, 90, 50],
            'score': [None, None, None],
            'strand': ['+', '+', '+'],
            'phase': [None, None, None],
            'ID': ['gene1', 'mrna1', None],
            'Name': ['test_gene', None, None],
            'Parent': [None, 'gene1', 'mrna1'],
            '_REF': ['ref', 'ref', 'ref'],
            '_QUERY': ['query', 'query', 'query']
        })

        result_df = gff_to_df(file_path)
        pd.testing.assert_frame_equal(result_df, expected_df)

    def test_gff_to_df_empty(self):
        """Test with an empty GFF file."""
        empty_gff_content = "##gff-version   3"  # Minimal content
        file_path = os.path.join(self.test_dir, "empty.gff")
        self.create_dummy_gff(file_path, empty_gff_content)

        result_df = gff_to_df(file_path)
        self.assertIsNone(result_df)

    def test_gff_to_df_not_found(self):
        """Test when the GFF file is not found."""
        file_path = os.path.join(self.test_dir, "nonexistent.gff")

        # Capture the print output
        import sys
        old_stdout = sys.stdout
        sys.stdout = captured_output = io.StringIO()

        result_df = gff_to_df(file_path)

        sys.stdout = old_stdout  # Restore stdout

        self.assertIsNone(result_df)
        self.assertIn("Error: GFF file not found:", captured_output.getvalue())

    def test_gff_to_df_invalid_format(self):
        """Test with an invalid GFF format."""
        invalid_gff_content = """chr1\t.\tgene\t1\t100\t.\t+\t.\tID=gene1"""  # Missing columns
        file_path = os.path.join(self.test_dir, "invalid.gff")
        self.create_dummy_gff(file_path, invalid_gff_content)

        result_df = gff_to_df(file_path)
        self.assertIsNone(result_df)

    def test_gff_to_df_one_line(self):
         """Test with a GFF file containing only one line (plus header)."""
         one_line_gff_content = """##gff-version   3\nchr1\ttest\tgene\t1\t100\t.\t+\t.\tID=gene1;Name=test_gene"""
         file_path = os.path.join(self.test_dir, "oneline.gff")
         self.create_dummy_gff(file_path, one_line_gff_content)

         result_df = gff_to_df(file_path)
         self.assertIsNone(result_df)


# Run the tests if the script is executed
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

**Explanation:**

1.  **Import Necessary Libraries:**
    * `unittest`:  The standard Python testing framework.
    * `pandas`:  For creating expected DataFrames and using `assert_frame_equal`.
    * `os`:  For creating and managing temporary test files and directories.
    * `gff_to_df`:  The function you want to test.
    * `io`:  For capturing standard output (used to test print statements).

2.  **`TestGffToDf` Class:**
    * This class inherits from `unittest.TestCase` and contains the test methods.
    * `setUp(self)`:  This method is executed before each test. Here, it creates a temporary directory `temp_test_dir` to hold test GFF files.
    * `tearDown(self)`:  This method is executed after each test. It cleans up by removing all files in the temporary directory and then the directory itself. This ensures tests don't interfere with each other and leaves no leftover files.
    * `create_dummy_gff(self, file_path, content)`: A helper method to create GFF files with specified content.  This reduces code duplication.

3.  **Test Methods:**
    * Each method name starts with `test_`.  The `unittest` framework recognizes these as test cases.
    * `test_gff_to_df_valid(self)`:
        * Creates a valid GFF file (`valid_gff_content`).
        * Defines the `expected_df` DataFrame that should be the result of parsing this GFF.  It's crucial to get this DataFrame exactly right!
        * Calls `gff_to_df` to get the `result_df`.
        * Uses `pd.testing.assert_frame_equal` to compare the `result_df` with the `expected_df`.  This method gives a detailed comparison, highlighting any differences in rows, columns, or data types.
    * `test_gff_to_df_empty(self)`:
        * Creates an empty GFF file.
        * Checks that `gff_to_df` returns `None` for an empty file.
    * `test_gff_to_df_not_found(self)`:
        * Tries to process a non-existent file.
        * Uses `io.StringIO` to capture what is printed to standard output (`print` statements in your function).  This allows you to test that the error message is printed as expected.
        * Asserts that `gff_to_df` returns `None` and that the captured output contains the "File not found" error message.
    * `test_gff_to_df_invalid_format(self)`:
        * Creates a GFF file with an invalid format (missing columns).
        * Asserts that `gff_to_df` returns `None` when the format is invalid.
    * `test_gff_to_df_one_line(self)`:
        * Creates a GFF file with only one data line.
        * Asserts that `gff_to_df` returns `None` for single-line input.

4.  **`if __name__ == '__main__':` Block:**
    * This ensures that the tests are run only when the script is executed directly (not when imported as a module).
    * `unittest.main(argv=['first-arg-is-ignored'], exit=False)`:  This runs the tests.  
        * `argv=['first-arg-is-ignored']` is a bit of a quirk needed to make `unittest.main` work correctly within a Jupyter Notebook.
        * `exit=False` is important!  It prevents `unittest.main` from calling `sys.exit()`, which would halt the Jupyter Notebook kernel.

**How to Use in a Jupyter Notebook:**

1.  **Save the Code:** Save the unittest code above as a Python file (e.g., `test_gff_to_df.py`) in the same directory as your `gff_to_df.py`.
2.  **Run in a Notebook Cell:** In a Jupyter Notebook cell, execute the following:

In [ ]:
import unittest
    import sys

    # Load the test file as a module
    loader = unittest.TestLoader()
    suite = loader.discover('.', pattern='test_gff_to_df.py')  # Adjust pattern if needed

    # Create a TestRunner that doesn't exit
    runner = unittest.TextTestRunner(verbosity=2, stream=sys.stdout)  # Use sys.stdout to print to notebook
    result = runner.run(suite)

    # Optionally, inspect the result object for more details
    print(result)

**Important Notes:**

* **Error Handling:** The tests cover the error handling in your `gff_to_df` function (file not found, empty file, invalid format).
* **Edge Cases:** The `test_gff_to_df_one_line` test addresses an edge case in your function's logic.
* **Readability:** The test code is well-commented to explain the purpose of each test.
* **Maintainability:** Using `setUp` and `tearDown` makes the tests cleaner and easier to maintain.  If you add more tests, you won't have to repeat the file creation and cleanup logic.
* **Comprehensive Testing:** This suite covers the main scenarios and potential issues in your function. You might want to add more tests for very complex GFF files or specific edge cases you anticipate.